In [1]:
from pathlib import Path
import sys

import iotbx.reflection_file_reader as cif_input

sys.path.append(str(Path(Path.home(), "Documents/xray/src")))
from miller_ops import get_miller_array
sys.path.append(str(Path(Path.home(), "Documents/xray/data/cifs/scripts")))
from generate_fmodel import write_cif, get_status_array

In [16]:
cif_dir = Path(Path.home(), "Documents/xray/dev/29_synthetic_native_3/data/cifs/7mhf_30_05/0")
new_cif_dir = Path(Path.home(), "Documents/xray/dev/38_standard_flags/data/bench/0")
cif_names = ["0.cif", "1.cif"]

In [17]:
f_obs_file = Path(cif_dir, cif_names[0])
f_obs_array = get_miller_array(
    f_obs_file=f_obs_file,
    label="_refln.F_meas_au"
)
f_obs_array.show_summary()

Miller array info: /Users/matthew/Documents/xray/dev/29_synthetic_native_3/data/cifs/7mhf_30_05/0/0.cif:Global,_refln.F_meas_au
Observation type: xray.amplitude
Type of data: double, size=147529
Type of sigmas: None
Number of Miller indices: 147529
Anomalous flag: False
Unit cell: (114.968, 54.622, 45.194, 90, 101.675, 90)
Space group: C 1 2 1 (No. 5)


crystal.symmetry(
    unit_cell=(114.968, 54.622, 45.194, 90, 101.675, 90),
    space_group_symbol="C 1 2 1"
  )
Global,_refln.F_meas_au
size: 147529

In [18]:
f_complete_array = f_obs_array.complete_set(d_min=1, d_max=100)
f_complete_array.show_summary()

Number of Miller indices: 147529
Anomalous flag: False
Unit cell: (114.968, 54.622, 45.194, 90, 101.675, 90)
Space group: C 1 2 1 (No. 5)


crystal.symmetry(
    unit_cell=(114.968, 54.622, 45.194, 90, 101.675, 90),
    space_group_symbol="C 1 2 1"
  )

In [19]:
f_complete_flags = f_complete_array.generate_r_free_flags(fraction=0.05, max_free=20000)
f_complete_flags.show_summary()

print(f_complete_array.select(f_complete_flags.data()).size())


Miller array info: None
Observation type: None
Type of data: bool, size=147529
Type of sigmas: None
Number of Miller indices: 147529
Anomalous flag: False
Unit cell: (114.968, 54.622, 45.194, 90, 101.675, 90)
Space group: C 1 2 1 (No. 5)
7377


In [20]:
f_complete_flags.indices()[0]

(-114, 0, 4)

In [21]:
f_complete_flags.at_first_index(f_complete_flags.data(), (-113, 1, 6))

False

In [22]:
for cif_name in cif_names:
    f_obs_file = Path(cif_dir, cif_name)
    f_obs_array = get_miller_array(
        f_obs_file=f_obs_file,
        label="_refln.F_meas_au"
    )
    status_array = get_miller_array(
        f_obs_file=f_obs_file,
        label="_refln.status"
    )
    flags_array = status_array.customized_copy(data=status_array.data()=="f")

    print(flags_array.select(flags_array.data()).size() / flags_array.size())

    # flags_array = f_obs_array.generate_r_free_flags()

    for index in f_obs_array.indices():
        # print(index)

        index_std_flag = f_complete_flags.at_first_index(f_complete_flags.data(), index)
        index_id = flags_array.first_index(index)
        flags_array.data()[index_id] = index_std_flag
        # print(index, index_std_flag)

    # print fraction
    print(flags_array.select(flags_array.data()).size() / flags_array.size())

    status_array = get_status_array(flags_array)
    cif_file = Path(new_cif_dir, cif_name)
    write_cif(f_obs=f_obs_array, status_array=status_array, cif_file=cif_file)

0.05000372808058077
0.05000372808058077
0.05000372808058077
0.05000372808058077


In [35]:
flags_array.first_index((-73, 1, 4))

0

In [34]:
flags_array.indices()[0]

(-73, 1, 4)